In [2]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.api import SimpleExpSmoothing
from math import sqrt
from df_setup import load_df

# ---------- PARAMETERS (you can tweak) ----------
SERVICE_LEVEL = 0.95            # target service level (0.95 = 95%)
Z = 1.645                       # z-score for 95% (change if SERVICE_LEVEL differs)
MIN_HISTORY_DAYS = 30           # skip combos with less history
FORECAST_METHOD = "SES"         # 'SES' or 'MA' (we'll use SES by default)
# ------------------------------------------------

# load raw (not aggregated) data so we can use Sales_Volume, Lead time, Stock_Level etc.
# make sure df_setup.load_df supports aggregate=False; if not, read CSV directly.
df_raw = pd.read_csv(r'C:\Users\Ankit Kumar Singh\Downloads\extended_fmcg_demand_forecasting.csv')

# Ensure Date column dtype
df_raw['Date'] = pd.to_datetime(df_raw['Date'])

# Final results list
final_rows = []

# Loop over product_category x store_location
for category in df_raw['Product_Category'].unique():
    for store in df_raw['Store_Location'].unique():
        sub = df_raw[(df_raw['Product_Category'] == category) & (df_raw['Store_Location'] == store)].copy()
        if sub.empty:
            continue

        # Sort & set index
        sub = sub.sort_values('Date').set_index('Date')

        # Make regular daily index (fill missing days with 0 sales)
        sub = sub.asfreq('D').fillna({'Sales_Volume': 0})

        # Need enough history
        if len(sub) < MIN_HISTORY_DAYS:
            continue

        # Use the most common (median) replenishment lead time for this group (in days)
        if 'Replenishment_Lead_Time' in sub.columns:
            lead_time = int(sub['Replenishment_Lead_Time'].median())
            if lead_time < 1:
                lead_time = 1
        else:
            lead_time = 7  # fallback default

        # Current stock (latest non-null Stock_Level)
        current_stock = None
        if 'Stock_Level' in sub.columns:
            nonnull = sub['Stock_Level'].dropna()
            current_stock = int(nonnull.iloc[-1]) if len(nonnull) > 0 else 0
        else:
            current_stock = 0

        # Historical daily demand series
        demand = sub['Sales_Volume']

        # Estimate daily mean and std dev
        mean_d = demand.mean()
        std_d = demand.std(ddof=0)  # population std (ddof=0). use ddof=1 if preferred

        # Forecast demand over lead time
        # Use SES on full history for forecast (if SES fails, fallback to mean*LT)
        try:
            ses = SimpleExpSmoothing(demand, initialization_method='estimated').fit()
            forecast_lt = ses.forecast(lead_time)  # returns series indexed by future dates
            forecast_during_lt = float(forecast_lt.sum())
        except Exception:
            # fallback: mean * lead_time
            forecast_during_lt = float(mean_d * lead_time)

        # Safety Stock (std during lead time)
        # sigma_LT = sigma_daily * sqrt(lead_time)
        sigma_LT = std_d * sqrt(lead_time)

        safety_stock = Z * sigma_LT

        # Reorder Point
        rop = forecast_during_lt + safety_stock

        # Suggested Order Quantity: bring inventory up to ROP (or to target level)
        suggested_order_qty = max(0, rop - current_stock)

        final_rows.append({
            'Product_Category': category,
            'Store_Location': store,
            'Lead_Time_days': lead_time,
            'Forecast_Demand_LT': round(forecast_during_lt, 2),
            'Daily_Mean_Demand': round(mean_d, 3),
            'Daily_StdDev_Demand': round(std_d, 3),
            'Safety_Stock': round(safety_stock, 2),
            'ROP': round(rop, 2),
            'Current_Stock': int(current_stock),
            'Suggested_Order_Qty': int(round(suggested_order_qty))
        })

# Create final DataFrame and save to Excel
final_df = pd.DataFrame(final_rows)
final_df = final_df.sort_values(['Product_Category', 'Store_Location']).reset_index(drop=True)

output_excel = "inventory_decision_table.xlsx"
final_df.to_excel(output_excel, index=False)

print("✅ Inventory decision table saved to:", output_excel)
final_df.head(20)


✅ Inventory decision table saved to: inventory_decision_table.xlsx


,Product_Category,Store_Location,Lead_Time_days,Forecast_Demand_LT,Daily_Mean_Demand,Daily_StdDev_Demand,Safety_Stock,ROP,Current_Stock,Suggested_Order_Qty
0,Beverages,Rural,5,413.39,82.678,324.394,1193.23,1606.62,443,1164
1,Beverages,Suburban,5,336.00,67.200,289.091,1063.37,1399.37,170,1229
2,Beverages,Urban,6,441.72,88.256,340.932,1373.76,1815.48,188,1627
3,Dairy,Rural,5,466.35,72.017,292.622,1076.36,1542.72,200,1343
4,Dairy,Suburban,5,430.53,86.106,331.451,1219.19,1649.72,152,1498
5,Dairy,Urban,4,205.88,51.469,260.293,856.36,1062.24,178,884
6,Household,Rural,4,316.59,79.147,304.052,1000.33,1316.92,180,1137
7,Household,Suburban,5,317.81,63.561,271.555,998.87,1316.67,143,1174
8,Household,Urban,5,362.77,72.556,308.974,1136.51,1499.28,241,1258
9,Personal Care,Rural,5,315.11,63.022,285.242,1049.22,1364.33,182,1182
